# Imports

In [1]:
# add to sys.path
import sys

sys.path.insert(0, "../")
sys.path.insert(0, "../src")

In [2]:
import time
import logging
import pandas as pd

from datetime import timedelta, datetime as dt, timezone
from dateutil.relativedelta import relativedelta
import plaid


# gcp
from google.cloud import bigquery
from google.cloud import pubsub_v1
from google.cloud import scheduler_v1
from google.cloud import functions_v2
from google.cloud.functions_v1.types import ListFunctionsRequest
from google.protobuf import duration_pb2

# SCHEMAS
from jobs.bq_table_schemas import BqTableSchemas
from jobs.cloud_schemas import CloudSchemas


# UTILS
from utils.bq_utils import BqUtils
from utils.plaid_utils import PlaidUtils
from utils.secrets_utils import SecretsUtils
from utils.budget_values import BudgetValues
from utils.financial_accounts import FinancialAccounts
from utils.plaid_transactions import PlaidTransactions
from utils.plaid_investments import PlaidInvestments


# get all secrets
sec = SecretsUtils()
secrets = sec.create_secrets_dict(secret_type="DEV")
PLAID_CLIENT_ID = secrets["PLAID_CLIENT_ID"]
PLAID_SECRET = secrets["PLAID_SECRET_DEV"]
PLAID_ACCESS_TOKENS = sec.get_access_token_secrets(secrets)
PLAID_HOST = plaid.Environment.Development
PLAID_PRODUCTS = ["liabilities", "transactions", "investments"]
PLAID_COUNTRY_CODES = ["US"]

# initialize clients
bq_client = bigquery.Client()
bq = BqUtils(bq_client=bq_client)
plaid_client = PlaidUtils(bq_client, PLAID_CLIENT_ID, PLAID_SECRET, PLAID_HOST)
financial_accounts = FinancialAccounts(bq_client, plaid_client)
plaid_transactions = PlaidTransactions(bq_client, plaid_client)
plaid_investments = PlaidInvestments(bq_client, plaid_client)
budget_values = BudgetValues(bq_client)
bq_tables = BqTableSchemas()

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 1000)

# Budget Values

In [3]:
# for i in range(-19, 1):
#     # print(bq.get_partition_date(i, partition_format="YYYYMM"))
#     budget_values.upload_budget_values_df_to_bq(offset=i)

# Access Tokens

In [4]:
# query = """
# SELECT
#     table_name,
# FROM `zsc-personal.personal_finance.INFORMATION_SCHEMA.TABLES`
# WHERE SUBSTR(table_name, -8) >= "20240526"
# """
# df = bq.query(query)

# tables = list(df["full_table_name"].unique())

# print(tables)

In [5]:
# bq.delete_list_of_tables(project_id="zsc-personal", dataset_id="personal_finance", table_ids=tables, confirm=True)

# Chase

In [6]:
hi = plaid_client.get_accounts(secrets["PLAID_TOKEN_CHASE"])

ERROR:root:
Status Code: 400
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Server': 'nginx', 'Date': 'Sat, 25 May 2024 16:11:52 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '387', 'Connection': 'keep-alive', 'plaid-version': '2020-09-14'})
HTTP response body: {
  "display_message": null,
  "error_code": "ITEM_LOGIN_REQUIRED",
  "error_message": "the login details of this item have changed (credentials, MFA, or required user action) and a user login is required to update this information. use Link's update mode to restore the item to a good state",
  "error_type": "ITEM_ERROR",
  "request_id": "WOytkGDpFwdLpLQ",
  "suggested_action": null
}

